In [ ]:
import math
import matplotlib.pyplot as plt
import time

M_PI = 3.14159
NUM_THETA_CELLS = 90

class AGV_State:
    def __init__(self,x,y,theta,g,f):
        self.x=x
        self.y=y
        self.theta=theta
        self.g=g
        #self.h=h
        self.f=f
        #self.vr=vr
        #self.vl=vl
        
class Trolley:
    def __init__(self,x,y,theta):
        self.x=x
        self.y=y
        self.theta = theta
"""        
class Helper:
    def __init__(self):
        self.velocity = 10 #[m/s]
        self.wheel_radius = 0.05 #[m]
        self.distance_between_wheels = 0.2 #[m]
        self.dt = 1 #[s]
        self.d1 = 0.3 #[m]
"""
        
class Path:
    def __init__(self,closed,came_from,final):
        self.closed = closed
        self.came_from = came_from
        self.final = final
        
   
    
class Global_Planner:
    def __init__(self,grid,start_x,start_y,start_theta,goal_x,goal_y):        
        #self.goal=State(goal_x,goal_y,0,0,0,0,0,0)
        #self.start=State(start_x,start_y,start_theta,0,0,0,0,0)
        self.grid = grid
        self.goal = [goal_x,goal_y]
        self.start = [start_x,start_y,start_theta]
        #self.variables = Helper()
        self.velocity = 10 # [m/s]
        self.wheel_radius = 0.05  #[m]
        self.distance_between_wheels = 0.5 #[m]0.2
        self.dt = 0.1 #[s]
        self.d1 = 2 #[m]0.3
    
    def get_augmented_vr_vl(self):
        v = 2 * self.velocity
        vr_vl_list = []
        for i in range(0,v,1):
            vr = i 
            vl = v-i
            #w= (self.wheel_radius / self.distance_between_wheels)*(vr-vl)
            #if
            vr_vl_list.append([vr,vl])
        self.vr_vl_list = vr_vl_list
    
    def heuristic(self,x,y):
        return abs(y-self.goal[1])+abs(x-self.goal[0]) #Manhattan distance
    
    def get_corners(self,state):
        #----------incomplete
        l=[int(state.x),int(state.y)]
        return [l]
        
        
    def check_for_boundary_collisions(self,agv_state,trolley_state):
        agv_sides = self.get_corners(agv_state)
        trolley_sides = self.get_corners(trolley_state)
        for val1 in agv_sides:
            for val2 in trolley_sides:
                if (val1[0]==val2[0] and val1[1]==val2[1]):
                    print("collision")
                    return False
        return True
        
    def expand(self,agv_state,trolley):
        next_states = []  
        trolley_states=[]
        self.get_augmented_vr_vl()
        for val in self.vr_vl_list:
            #linear and angular velocity calculations
            linear_velocity = (self.wheel_radius/2)*(val[0]+val[1])
            angular_velocity = (self.wheel_radius/self.distance_between_wheels)*(val[0]-val[1])
            #Calculation of next AGV state
            next_x = agv_state.x + linear_velocity * math.cos(agv_state.theta) * self.dt
            next_y = agv_state.y + linear_velocity * math.sin(agv_state.theta) * self.dt
            next_theta = (agv_state.theta + angular_velocity * self.dt)%360
            next_g = agv_state.g + 1
            next_f = next_g + self.heuristic(next_x,next_y)
            agv_state = AGV_State(next_x,next_y,next_theta,next_g,next_f)
            #Calculation of next Trolley state
            trolley_next_theta = trolley.theta + linear_velocity/self.d1 * math.sin(next_theta-trolley.theta)
            trolley_next_x = next_x - self.d1 * math.cos(trolley.theta)
            trolley_next_y = next_y - self.d1 * math.sin(trolley.theta)
            trolley_state = Trolley(trolley_next_x,trolley_next_y,trolley_next_theta)
            
            if(self.check_for_boundary_collisions(agv_state,trolley_state)):
                next_states.append(agv_state)
                trolley_states.append(trolley_state)
                
        return next_states,trolley_states
    
    def index(self,x):
        return int(math.floor(x))
        
    def theta_to_stack_number(self,theta):
        new_theta = math.fmod((theta + 2 * M_PI),(2 * M_PI));
        stack_number = int((round(new_theta * NUM_THETA_CELLS / (2*M_PI))) % NUM_THETA_CELLS)
        return stack_number;
               
    def search(self):
        opened = []
        opened_trolley = {}
        closed = [[[0 for x in range(len(self.grid[0]))] for y in range(len(self.grid))] for cell in range(NUM_THETA_CELLS)]
        came_from = [[[0 for x in range(len(self.grid[0]))] for y in range(len(self.grid))] for cell in range(NUM_THETA_CELLS)]
        closed_trolley = [[[0 for x in range(len(self.grid[0]))] for y in range(len(self.grid))] for cell in range(NUM_THETA_CELLS)]
        came_from_trolley = [[[0 for x in range(len(self.grid[0]))] for y in range(len(self.grid))] for cell in range(NUM_THETA_CELLS)]
        
        #----Initial AGV parameters
        x = self.start[0]
        y = self.start[1]
        theta = self.start[2]
        g = 0
        f = g + self.heuristic(self.start[0], self.start[1])
        state = AGV_State(x, y, theta, 0, f)
        opened.append(state)
        
        stack_number = self.theta_to_stack_number(state.theta)
        closed[stack_number][self.index(state.x)][self.index(state.y)] = 1
        came_from[stack_number][self.index(state.x)][self.index(state.y)] = state 
        
        #----Initial trolley parameters
        theta_trolley = theta
        x_trolley = x - self.d1*math.cos(theta)
        y_trolley = y - self.d1*math.sin(theta)
        state_trolley = Trolley(x_trolley,y_trolley,theta_trolley)
        opened_trolley[state] = state_trolley
        
        stack_number_trolley = self.theta_to_stack_number(theta_trolley)
        closed_trolley[stack_number_trolley][self.index(x_trolley)][self.index(y_trolley)] = 1
        came_from[stack_number_trolley][self.index(x_trolley)][self.index(y_trolley)] = state_trolley
        
        counter,counter_2 =1,1
        while(opened!=[]):
            #print("counter_",counter)
            opened.sort(key=lambda state:state.f)
            current_agv = opened.pop(0)
            current_trolley = opened_trolley[current_agv]
            
            if (self.index(current_agv.x) == self.goal[0]) and (self.index(current_agv.y) == self.goal[1]):
                print("Goal reached")
                final = current_agv
                final_trolley = current_trolley
                agv_path = Path(closed,came_from,final)
                trolley_path = Path(closed_trolley,came_from_trolley,final_trolley)
                return agv_path,trolley_path #reconstruct path of udacity
            
            next_states,trolley_states = self.expand(current_agv,current_trolley) #call of expand function to get next states
            #print("next_states:",next_states)
            #print("trolley_states:",trolley_states)
            for next_state,trolley_state in zip(next_states,trolley_states):
                if(next_state.x < 0 or trolley_state.x < 0 or next_state.x > len(self.grid) or trolley_state.x > len(self.grid) or next_state.y < 0 or trolley_state.y < 0 or next_state.y > len(self.grid[0]) or trolley_state.y > len(self.grid[0])):
                    continue
                #print("counter_2:",counter_2)
                stack_number_agv = self.theta_to_stack_number(next_state.theta)
                stack_number_trolley = self.theta_to_stack_number(trolley_state.theta)
                if closed[stack_number_agv][self.index(next_state.x)][self.index(next_state.y)] == 0 and self.grid[self.index(next_state.x)][self.index(next_state.y)] == 0 and closed[stack_number_trolley][self.index(trolley_state.x)][self.index(trolley_state.y)] == 0 and self.grid[self.index(trolley_state.x)][self.index(trolley_state.y)] == 0:
                    
                    opened.append(next_state)
                    closed[stack_number_agv][self.index(next_state.x)][self.index(next_state.y)] = 1
                    came_from[stack_number_agv][self.index(next_state.x)][self.index(next_state.y)] = current_agv
                    
                    opened_trolley[next_state] = trolley_state
                    closed_trolley[stack_number_trolley][self.index(trolley_state.x)][self.index(trolley_state.y)] = 1
                    came_from_trolley[stack_number_trolley][self.index(trolley_state.x)][self.index(trolley_state.y)] = current_trolley
                counter_2+=1
            counter+=1
        print("Goal cannot be reached")


        
    def reconstruct_path(self,path):
        path_list = [path.final]
        stack = self.theta_to_stack_number(path.final.theta)
        current = path.came_from[stack][self.index(path.final.x)][self.index(path.final.y)]
        stack = self.theta_to_stack_number(path.final.theta)
        x=current.x
        y=current.y
        while(x!=self.start[0] and y!=self.start[1]):
            path_list.append(current)
            current = path.came_from[stack][self.index(x)][self.index(y)]
            x=current.x
            y=current.y
            stack = self.theta_to_stack_number(current.theta)
        return path_list
        
        
                
            
if __name__ == '__main__':
    
    grid_x=70
    grid_y=70
    grid_reso = 0.5
    grid = [[0 for i in range(int(grid_x/grid_reso))] for j in range(int(grid_y/grid_reso))]
    start_time = time.time()
    print("start:",start_time)
    gp = Global_Planner(grid,3,3,0,60,60)    
    ap,tp = gp.search()
    #print("ap length:",len(ap))
    #print("tp length:",len(tp))
    agv_path_list = gp.reconstruct_path(ap)
    trolley_path_list = gp.reconstruct_path(tp)
    end_time = time.time()
    print("end:",end_time-start_time)
    ax,ay=[],[]
    tx,ty=[],[]
    for val in agv_path_list:
        ax.append(val.x)
        ay.append(val.y)
    for val in trolley_path_list:
        tx.append(val.x)
        ty.append(val.y)
    
    plt.plot(ax,ay,'.k')
    plt.plot(tx,ty,'.r')
    plt.grid(True)
    plt.axis("equal")
    plt.show()
    
    
    
    
    





        
        

start: 1536155450.0531254
collision
collision
collision
collision
collision
collision
collision
collision
collision
collision
collision
collision
collision
collision
collision
collision
collision
collision
collision
Goal reached
